## Problem statement

Consider the following optimization problem adapted from [Christiansen et al. (2020)](http://doi.org/10.1364/OE.28.004444):
We want to design a metallic (silver) nanoparticle to focus an incident $H_z$-polarized planewave on
a single spot, maximizing the electric-field intensity at this focal spot.   The metallic
structure can be *any shape* of *any topology* (any connectivity, number of holes, etcetera)
surrounding the focal spot, as long as the metal lies within an annular "design region" $\Omega_d$:
between a minimum radius $r_s = 10$nm (the minimum distance from the focal spot) and an outer
radius $r_d=100$nm.  The computational cell is of height $H$ and length $L$, and we employ a perfectly matched layer (PML) thickness of $d_{pml}$ to implement outgoing (radiation) boundary conditions for this finite domain.
![](Illustration.png)

The goal is find the arrangement of the silver material in the gray region that maximizes the |electric field|² at the center (the focal point). Every "pixel" in the gray region is effectively treated as a degree of freedom that can vary continuously between silver (shown in black below) and air (shown in white below). This is called density-based [topology optimization (TO)](https://en.wikipedia.org/wiki/Topology_optimization), and leads to a tractable optimization problem despite the huge number of parameters. A standard "projection" technique, described below, is used to "binarize" the structure by eventually forcing the material to be either silver or air almost everywhere.


## Topology optimization

We use density-based topology optimization (TO) to maximize the electric field intensity at the center. In TO, every point in the design domain is a design degree of freedom that can vary continuously between air ($p=0$) and silver ($p=1$), which we discretize into a piece-wise constant parameter space $P$ for the design parameter $p\in [0,1]$. The material's electric permittivity ε is then given by:

$$
\varepsilon(p) = \left[n_{air}+p(n_{metal}-n_{air})\right]^2, 
$$
where $n_{air}=1$ and $n_{metal}$ are the refractive indices ($\sqrt{\varepsilon}$) of the air and metal, respectively. (It is tempting to simply linearly interpolate the permittivities ε, rather than the refractive indices, but this turns out to lead to artificial singularities in the case of metals where ε can pass through zero [4].) 

In practice, to avoid obtaining arbitrarily fine features as the spatial resolution is increased, one needs to regularize the problem with a minimum lengthscale $r_f$ by generating a smoothed/filtered parameter function $p_f$.  (Although this regularizes the problem, strictly speaking it does not impose a minimum feature size because of the nonlinear-projection step below. In practical applications, one imposes additional [manufacturing constraints](http://doi.org/10.1364/OE.431188) explicitly.)  We perform the smoothing $p \to p_f$ by solving a simple "damped diffusion" PDE, also called a Helmholtz filter [5], for $p_f$ given the design variables $p$: 
$$
\begin{aligned}    
-r_f^2\nabla^2p_f+p_f&=p\, ,\\
\left. \frac{\partial p_f}{\partial \vec{n}} \right\vert_{\partial\Omega_D} & =0 . 
\end{aligned} 
$$

We choose a filter radius $r_f=R_f/(2\sqrt{3})$ where $R_f=10$ nm, in order to match a published result (using a slightly different filtering scheme) for comparison [6]. 

Next, we apply a smoothed threshold projection to the intermediate variable $p_f$ to obtain a "binarized" density parameter $p_t$ that tends towards values of $0$ or $1$ almost everywhere [6] as the steepness $\beta$ of the thresholding is increased: 
$$
p_t = \frac{\tanh(\beta\eta)+\tanh\left[\beta(p_f-\eta)\right]}{\tanh(\beta\eta)+\tanh\left[\beta(1-\eta)\right]}. 
$$
Note that as $\beta\to\infty$, this threshold procedure goes to a step function, which would make the optimization problem non-differentiable. In consequence, the standard approach is to gradually increase $\beta$ to slowly binarize the design as the optimization progresses [6]. We will show how this is done below.  

Finally, we replace $p$ with the filtered and thresholded $p_t$ in the ε interpolation formula from above:

$$
\varepsilon(p_t) = \left[n_{air}+p_t(n_{metal}-n_{air})\right]^2,
$$
This is the quantity that will be used for the $1/\varepsilon(x)$ coefficient in our Helmholtz PDE.


## Coding
The following code is used to obtain the optimized geometry and the corresponding field, a more detailed explanation of the code can be found in this [tutorial](https://gridap.github.io/Tutorials/dev/pages/t018_TopOptEMFocus/).

In [ ]:
using Gridap, Gridap.Geometry, Gridap.Fields, GridapGmsh
using LinearAlgebra
using NLopt
using CairoMakie, GridapMakie
using ChainRulesCore, Zygote
import ChainRulesCore: rrule
NO_FIELDS = ZeroTangent()

λ = 532      # Wavelength (nm)
L = 600      # Width of the numerical cell (excluding PML) (nm)
h1 = 600     # Height of the air region below the source (nm)
h2 = 200     # Height of the air region above the source (nm)
dpml = 300   # Thickness of the PML (nm)

n_metal = 0.054 + 3.429im # Silver refractive index at λ = 532 nm
n_air = 1    # Air refractive index
μ = 1        # Magnetic permeability
k = 2*π/λ    # Wavenumber (nm^-1)


model = GmshDiscreteModel("RecCirGeometry.msh")

order = 1
reffe = ReferenceFE(lagrangian, Float64, order)
V = TestFESpace(model, reffe, dirichlet_tags = ["DirichletEdges", "DirichletNodes"], vector_type = Vector{ComplexF64})
U = V   # mathematically equivalent to TrialFESpace(V,0)

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_s = BoundaryTriangulation(model; tags = ["Source"]) # Source line 
dΓ_s = Measure(Γ_s, degree)

Ω_d = Triangulation(model, tags="Design")
dΩ_d = Measure(Ω_d, degree)

Ω_c = Triangulation(model, tags="Center")
dΩ_c = Measure(Ω_c, degree)

p_reffe = ReferenceFE(lagrangian, Float64, 0)
Q = TestFESpace(Ω_d, p_reffe, vector_type = Vector{Float64})
P = Q
np = num_free_dofs(P) # Number of cells in design region (number of design parameters)

pf_reffe = ReferenceFE(lagrangian, Float64, 1)
Qf = TestFESpace(Ω_d, pf_reffe, vector_type = Vector{Float64})
Pf = Qf
fem_params = (; V, U, Q, P, Qf, Pf, np, Ω, dΩ, dΩ_d, dΩ_c, dΓ_s)


R = 1e-10
LHp=(L/2, h1+h2)   # Start of PML for x,y > 0
LHn=(L/2, 0)       # Start of PML for x,y < 0
phys_params = (; k, n_metal, n_air, μ, R, dpml, LHp, LHn)

# PML coordinate streching functions
function s_PML(x; phys_params)
    σ = -3 / 4 * log(phys_params.R) / phys_params.dpml / phys_params.n_air
    xf = Tuple(x)
    u = @. ifelse(xf > 0 , xf - phys_params.LHp, - xf - phys_params.LHn)
    return @. ifelse(u > 0,  1 + (1im * σ / phys_params.k) * (u / phys_params.dpml)^2, $(1.0+0im))
end

function ds_PML(x; phys_params)
    σ = -3 / 4 * log(phys_params.R) / phys_params.dpml / phys_params.n_air
    xf = Tuple(x)
    u = @. ifelse(xf > 0 , xf - phys_params.LHp, - xf - phys_params.LHn)
    ds = @. ifelse(u > 0, (2im * σ / phys_params.k) * (1 / phys_params.dpml)^2 * u, $(0.0+0im))
    return ds.*sign.(xf)
end

struct Λ{PT} <: Function
    phys_params::PT
end

function (Λf::Λ)(x)
    s_x,s_y = s_PML(x; Λf.phys_params)
    return VectorValue(1/s_x, 1/s_y)
end

# Define the derivative for the Λ factor
Fields.∇(Λf::Λ) = x -> TensorValue{2, 2, ComplexF64}(-(Λf(x)[1])^2 * ds_PML(x; Λf.phys_params)[1], 0, 0, -(Λf(x)[2])^2 * ds_PML(x; Λf.phys_params)[2])


r = 5/sqrt(3)               # Filter radius
β = 32.0                    # β∈[1,∞], threshold sharpness
η = 0.5                     # η∈[0,1], threshold center    

a_f(r, u, v) = r^2 * (∇(v) ⋅ ∇(u))

function Filter(p0; r, fem_params)
    ph = FEFunction(fem_params.P, p0)
    op = AffineFEOperator(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ_d + ∫(v * u)fem_params.dΩ_d, ∫(v * ph)fem_params.dΩ_d
      end
    pfh = solve(op)
    return get_free_dof_values(pfh)
end

function Threshold(pfh; β, η)
    return ((tanh(β * η) + tanh(β * (pfh - η))) / (tanh(β * η) + tanh(β * (1.0 - η))))         
end


ξd(p, n_air, n_metal)= 1 / (n_air + (n_metal - n_air) * p)^2 - 1 / n_air^2 # in the design region

a_base(u, v; phys_params) = (1 / phys_params.n_air^2) * ((∇ .* (Λ(phys_params) * v)) ⊙ (Λ(phys_params) .* ∇(u))) - (phys_params.k^2 * phys_params.μ * (v * u))

a_design(u, v, pth; phys_params) = ((p -> ξd(p, phys_params.n_air, phys_params.n_metal)) ∘ pth) * (∇(v) ⊙ ∇(u))

function MatrixA(pth; phys_params, fem_params)
    A_mat = assemble_matrix(fem_params.U, fem_params.V) do u, v
        ∫(a_base(u, v; phys_params))fem_params.dΩ + ∫(a_design(u, v, pth; phys_params))fem_params.dΩ_d
    end
    return lu(A_mat)
end


function MatrixOf(fem_params)
    x0 = VectorValue(0,300)  # Position of the field to be optimized
    δ = 1                    
    return assemble_matrix(fem_params.U, fem_params.V) do u, v
        ∫((x->(1/(2*π)*exp(-norm(x - x0)^2 / 2 / δ^2))) * (∇(u) ⋅ ∇(v)) )fem_params.dΩ_c
    end
end


Dptdpf(pf, β, η) = β * (1.0 - tanh(β * (pf - η))^2) / (tanh(β * η) + tanh(β * (1.0 - η)))

Dξdpf(pf, n_air, n_metal, β, η)= 2 * (n_air - n_metal) / (n_air + (n_metal - n_air) * Threshold(pf; β, η))^3 * Dptdpf(pf, β, η)

DAdpf(u, v, pfh; phys_params, β, η) = ((p -> Dξdpf(p, phys_params.n_air, phys_params.n_metal, β, η)) ∘ pfh) * (∇(v) ⊙ ∇(u))

function gf_pf(pf_vec; β, η, phys_params, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; phys_params, fem_params)
    b_vec = assemble_vector(v->(∫(v)fem_params.dΓ_s), fem_params.V)
    u_vec = A_mat \ b_vec
    
    O_mat = MatrixOf(fem_params)
    real(u_vec' * O_mat * u_vec)
end

function rrule(::typeof(gf_pf), pf_vec; β, η, phys_params, fem_params)
    function U_pullback(dgdg)
      NO_FIELDS, dgdg * Dgfdpf(pf_vec; β, η, phys_params, fem_params)
    end
    gf_pf(pf_vec; β, η, phys_params, fem_params), U_pullback
end

function Dgfdpf(pf_vec; β, η, phys_params, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; phys_params, fem_params)
    b_vec = assemble_vector(v->(∫(v)fem_params.dΓ_s), fem_params.V)
    u_vec = A_mat \ b_vec
    O_mat = MatrixOf(fem_params)
    
    uh = FEFunction(fem_params.U, u_vec)
    w_vec =  A_mat' \ (O_mat * u_vec)
    wconjh = FEFunction(fem_params.U, conj(w_vec)) 
    
    l_temp(dp) = ∫(real(-2 * DAdpf(uh, wconjh, pfh; phys_params, β, η)) * dp)fem_params.dΩ_d
    dgfdpf = assemble_vector(l_temp, fem_params.Pf)
    return dgfdpf
end

function pf_p0(p0; r, fem_params)
    pf_vec = Filter(p0; r, fem_params)
    pf_vec[pf_vec .< 0] .= 0
    pf_vec[pf_vec .> 1] .= 1.0
    pf_vec
end

function rrule(::typeof(pf_p0), p0; r, fem_params)
  function pf_pullback(dgdpf)
    NO_FIELDS, Dgdp(dgdpf; r, fem_params)
  end
  pf_p0(p0; r, fem_params), pf_pullback
end

function Dgdp(dgdpf; r, fem_params)
    Af = assemble_matrix(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ_d + ∫(v * u)fem_params.dΩ_d
    end
    wvec = Af' \ dgdpf
    wh = FEFunction(fem_params.Pf, wvec)
    l_temp(dp) = ∫(wh * dp)fem_params.dΩ_d
    return assemble_vector(l_temp, fem_params.P)
end

function gf_p(p0::Vector; r, β, η, phys_params, fem_params)
    pf_vec = pf_p0(p0; r, fem_params)
    gf_pf(pf_vec; β, η, phys_params, fem_params)
end

function gf_p(p0::Vector, grad::Vector; r, β, η, phys_params, fem_params)
    if length(grad) > 0
        dgdp, = Zygote.gradient(p -> gf_p(p; r, β, η, phys_params, fem_params), p0)
        grad[:] = dgdp
    end
    gvalue = gf_p(p0::Vector; r, β, η, phys_params, fem_params)
    open("gvalue.txt", "a") do io
        write(io, "$gvalue \n")
    end
    gvalue
end


function gf_p_optimize(p_init; r, β, η, TOL = 1e-4, MAX_ITER = 500, phys_params, fem_params)
    ##################### Optimize #################
    opt = Opt(:LD_MMA, fem_params.np)
    opt.lower_bounds = 0
    opt.upper_bounds = 1
    opt.ftol_rel = TOL
    opt.maxeval = MAX_ITER
    opt.max_objective = (p0, grad) -> gf_p(p0, grad; r, β, η, phys_params, fem_params)

    (g_opt, p_opt, ret) = optimize(opt, p_init)
    @show numevals = opt.numevals # the number of function evaluations
    return g_opt, p_opt
end

p_opt = fill(0.4, fem_params.np)   # Initial guess
β_list = [8.0, 16.0, 32.0]

g_opt = 0
TOL = 1e-8
MAX_ITER = 100
for bi = 1 : 3
    β = β_list[bi]
    g_opt, p_temp_opt = gf_p_optimize(p_opt; r, β, η, TOL, MAX_ITER, phys_params, fem_params)
    global p_opt = p_temp_opt
end
@show g_opt

p0 = p_opt

pf_vec = pf_p0(p0; r, fem_params)
pfh = FEFunction(fem_params.Pf, pf_vec)
pth = (pf -> Threshold(pf; β, η)) ∘ pfh
A_mat = MatrixA(pth; phys_params, fem_params)
b_vec = assemble_vector(v->(∫(v)fem_params.dΓ_s), fem_params.V)
u_vec = A_mat \ b_vec
uh = FEFunction(fem_params.U, u_vec)

fig, ax, plt = plot(fem_params.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(1)
ax.title = "Design Shape"
rplot = 110 # Region for plot
limits!(ax, -rplot, rplot, (h1)/2-rplot, (h1)/2+rplot)
save("shape.png", fig)

maxe = 30 # Maximum electric field magnitude compared to the incident plane wave
e1=abs2(phys_params.n_air^2)   
e2=abs2(phys_params.n_metal^2)

fig, ax, plt = plot(fem_params.Ω, 2*(sqrt∘(abs((conj(∇(uh)) ⋅ ∇(uh))/(CellField(e1,fem_params.Ω) + (e2 - e1) * pth)))), colormap = :hot, colorrange=(0, maxe))
Colorbar(fig[1,2], plt)
ax.title = "|E|"
ax.aspect = AxisAspect(1)
limits!(ax, -rplot, rplot, (h1)/2-rplot, (h1)/2+rplot)
save("Field.png", fig)

## Results
The optimized geometry and the corresponding field are displayed below.
![](shape.png)
![](Field.png)

We can see that this shape is very close to that from [Christiansen et al. (2020)](http://doi.org/10.1364/OE.28.004444). In particular, the optimized field enhancement $|E/E_0|^2$ is found to be $4 \times$ `g_opt` = 700 at $\beta = 32$ (the factor 4 comes from the incident plane wave amplitude). For comparison, the optimized field enhancement from [Christiansen et al. (2020)](http://doi.org/10.1364/OE.28.004444) is about ~1000, that is because they did a further binarizing to the design parameters. Let $\beta=80$ and run for a 100 more optimization steps, we obtain $4 \times$ `g_opt` = 980 $\approx$ 1000, proving our results.

## Mesh generation
The following code is used to generate the computation cell mesh file using Gmsh. 

In [ ]:
using Gmsh
import Gmsh: gmsh

struct RecCirGeometry
    L::Float64           # Length of the normal region  
    h1::Float64          # Height of normal region
    h2::Float64          # Height of the region above source
    rt::Float64          # Radius of the target location
    rd::Float64          # Radius of design domain
    rs::Float64          # Radius of smallest distance circle
    dpml::Float64        # Thickness of the PML
    # Characteristic length (controls the resolution, smaller the finer)
    l1::Float64          # Normal region
    l2::Float64          # Design domain
end


function MeshGenerator(geo_param::RecCirGeometry, meshfile_name::String)
    gmsh.initialize()
    gmsh.option.setNumber("General.Terminal", 1)
    gmsh.option.setNumber("Mesh.Algorithm", 6)
    gmsh.clear()
    gmsh.model.add("geometry")

    # Add points
    gmsh.model.geo.addPoint(-geo_param.L/2-geo_param.dpml, -geo_param.dpml, 0, geo_param.l1, 1)
    gmsh.model.geo.addPoint( geo_param.L/2+geo_param.dpml, -geo_param.dpml, 0, geo_param.l1, 2)
    gmsh.model.geo.addPoint( geo_param.L/2+geo_param.dpml,  geo_param.h1, 0, geo_param.l1, 3)
    gmsh.model.geo.addPoint( geo_param.L/2+geo_param.dpml,  geo_param.h1+geo_param.h2+geo_param.dpml, 0, geo_param.l1, 4)
    gmsh.model.geo.addPoint(-geo_param.L/2-geo_param.dpml,  geo_param.h1+geo_param.h2+geo_param.dpml, 0, geo_param.l1, 5)
    gmsh.model.geo.addPoint(-geo_param.L/2-geo_param.dpml,  geo_param.h1, 0, geo_param.l1, 6)
    gmsh.model.geo.addPoint(0,  geo_param.h1/2, 0, geo_param.l2, 7)
    gmsh.model.geo.addPoint(-geo_param.rs,  geo_param.h1/2, 0, geo_param.l2, 8)
    gmsh.model.geo.addPoint( geo_param.rs,  geo_param.h1/2, 0, geo_param.l2, 9)
    gmsh.model.geo.addPoint(-geo_param.rd,  geo_param.h1/2, 0, geo_param.l2, 10)
    gmsh.model.geo.addPoint( geo_param.rd,  geo_param.h1/2, 0, geo_param.l2, 11)
    # Add lines
    gmsh.model.geo.addLine( 1,  2,  1)
    gmsh.model.geo.addLine( 2,  3,  2)
    gmsh.model.geo.addLine( 3,  4,  3)
    gmsh.model.geo.addLine( 4,  5,  4)
    gmsh.model.geo.addLine( 6,  5,  5)
    gmsh.model.geo.addLine( 1,  6,  6)
    gmsh.model.geo.addLine( 6,  3,  7)
    gmsh.model.geo.addCircleArc( 8, 7, 9, 8)
    gmsh.model.geo.addCircleArc( 9, 7, 8, 9)
    gmsh.model.geo.addCircleArc(10, 7,11,10)
    gmsh.model.geo.addCircleArc(11, 7,10,11)
    # Construct curve loops and surfaces 
    gmsh.model.geo.addCurveLoop([8, 9], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.addCurveLoop([10, 11], 2)
    gmsh.model.geo.addPlaneSurface([2, 1], 2)
    gmsh.model.geo.addCurveLoop([1, 2, -7,-6], 4)
    gmsh.model.geo.addPlaneSurface([4, 2], 4)
    gmsh.model.geo.addCurveLoop([3, 4,-5, 7], 5)
    gmsh.model.geo.addPlaneSurface([5], 5)
    # Physical groups
    gmsh.model.addPhysicalGroup(0, [1, 2, 4, 5], 1)
    gmsh.model.setPhysicalName(0, 1, "DirichletNodes")
    gmsh.model.addPhysicalGroup(1, [1, 4], 2)
    gmsh.model.setPhysicalName(1, 2, "DirichletEdges")
    gmsh.model.addPhysicalGroup(0, [8, 9, 10, 11], 3)
    gmsh.model.setPhysicalName(0, 3, "DesignNodes")
    gmsh.model.addPhysicalGroup(1, [8, 9, 10, 11], 4)
    gmsh.model.setPhysicalName(1, 4, "DesignEdges")
    gmsh.model.addPhysicalGroup(2, [2], 5)
    gmsh.model.setPhysicalName(2, 5, "Design")
    gmsh.model.addPhysicalGroup(2, [1], 6)
    gmsh.model.setPhysicalName(2, 6, "Center")
    gmsh.model.addPhysicalGroup(2, [3,4,5], 7)
    gmsh.model.setPhysicalName(2, 7, "Air")
    gmsh.model.addPhysicalGroup(1, [7], 9)
    gmsh.model.setPhysicalName(1, 9, "Source")
    gmsh.model.geo.synchronize()
    
    # Set periodic mesh on the left and right side
    gmsh.model.mesh.setPeriodic(1, [2], [6],
                            [1, 0, 0, geo_param.L+2*geo_param.dpml, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])
    gmsh.model.mesh.setPeriodic(1, [3], [5],
                            [1, 0, 0, geo_param.L+2*geo_param.dpml, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])
    # We can then generate a 2D mesh...
    gmsh.model.mesh.generate(2)
    
    

    # ... and save it to disk
    gmsh.write(meshfile_name)
    gmsh.finalize()
end

L = 600
h1 = 600
h2 = 200
rt = 150
rd = 100
rs = 10

dpml = 300

l1 = 20
l2 = 1

meshfile = "RecCirGeometry.msh"
geo_param = RecCirGeometry(L, h1, h2, rt, rd, rs, dpml, l1, l2)
MeshGenerator(geo_param, meshfile)